In [1]:
import time
import numpy as np

In [2]:
import sc3nb as scn

# OSC communication
With the OSC communication module of sc3nb you can directly send and receive OSC packets.

Open Sound Control (OSC) is a networking protocol for sound and is used by SuperCollider to communicate between sclang and scsynth. sc3nb is itself a OSC client and server. This allows sc3nb to send and receive OSC traffic.

For more information on OSC and especially how Supercollider handles OSC packets please refer to the following links:

* [Open Sound Control Specification](http://opensoundcontrol.org/spec-1_0.html)
* [Server vs Client SuperCollider Guide](http://doc.sccode.org/Guides/ClientVsServer.html)
* [Server Command Reference](https://doc.sccode.org/Reference/Server-Command-Reference.html)
* [SuperCollider Synth Server Architecture](https://doc.sccode.org/Reference/Server-Architecture.html)

In [3]:
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang process... 

Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... 

Done.


sc3nb serves as OSC server and as client of the SuperCollider server `scsynth`.<br>
You can also communicate with the SuperCollider interpreter `sclang` via OSC.<br>


You can see the current connection information with `sc.server.connection_info()`

In [4]:
(sc3nb_ip, sc3nb_port), receivers = sc.server.connection_info()

This instance is at ('127.0.0.1', 57130),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 "sclang" at ('127.0.0.1', 57120)
                 


In [5]:
(sc3nb_ip, sc3nb_port), receivers

(('127.0.0.1', 57130),
 {('127.0.0.1', 57110): 'scsynth', ('127.0.0.1', 57120): 'sclang'})

If you want to communicate via OSC with another receiver you could add its name via `sc.server.add_receiver(name: str, ip: str, port: int)`, or you can pass a custom receiver when sending OSC

In [6]:
sc.server.add_receiver("sc3nb", sc3nb_ip, sc3nb_port)

In [7]:
sc.server.connection_info()

This instance is at ('127.0.0.1', 57130),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 "sclang" at ('127.0.0.1', 57120)
                 "sc3nb" at ('127.0.0.1', 57130)
                 


(('127.0.0.1', 57130),
 {('127.0.0.1', 57110): 'scsynth',
  ('127.0.0.1', 57120): 'sclang',
  ('127.0.0.1', 57130): 'sc3nb'})

## Sending OSC

You can send OSC with 

In [8]:
help(sc.server.send)

Help on method send in module sc3nb.osc.osc_communication:

send(package: Union[sc3nb.osc.osc_communication.OSCMessage, sc3nb.osc.osc_communication.Bundler], *, receiver: Union[str, Tuple[str, int], NoneType] = None, bundle: bool = False, await_reply: bool = True, timeout: float = 5) -> Any method of sc3nb.sc_objects.server.SCServer instance
    Sends OSC packet
    
    Parameters
    ----------
    package : OSCMessage or Bundler
        Object with `dgram` attribute.
    receiver : str or Tuple[str, int], optional
        Where to send the packet, by default send to default receiver
    bundle : bool, optional
        If True it is allowed to bundle the package with bundling, by default False.
    await_reply : bool, optional
        If True ask for reply from the server and return it,
        otherwise send the message and return None directly, by default True.
        If the package is bundled None will be returned.
    timeout : int, optional
        timeout in seconds for reply,

### Messages

Use the `OSCMessage` or the `python-osc` package to build a OscMessage

In [9]:
scn.OSCMessage?

In [10]:
msg = scn.OSCMessage("/s_new", ["s1", -1, 1, 1,])
sc.server.send(msg)

A shortcut for sending Messages is

In [11]:
help(sc.server.msg)

Help on method msg in module sc3nb.osc.osc_communication:

msg(msg_addr: str, msg_params: Union[Sequence, NoneType] = None, *, bundle: bool = False, receiver: Union[Tuple[str, int], NoneType] = None, await_reply: bool = True, timeout: float = 5) -> Union[Any, NoneType] method of sc3nb.sc_objects.server.SCServer instance
    Creates and sends OSC message over UDP.
    
    Parameters
    ----------
    msg_addr : str
        SuperCollider address of the OSC message
    msg_params : Optional[Sequence], optional
        List of paramters of the OSC message, by default None
    bundle : bool, optional
        If True it is allowed to bundle the content with bundling, by default False
    receiver : tuple[str, int], optional
        (IP address, port) to send the message, by default send to default receiver
    await_reply : bool, optional
        If True send message and wait for reply
        otherwise send the message and return directly, by default True
    timeout : float, optional
   

In [12]:
sc.server.msg("/s_new", ["s1", -1, 1, 1,])

a more complex example

In [13]:
for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
    freq = scn.midicps(60+p)  # see helper fns below
    sc.server.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "dur", 0.5, "num", 1])
    time.sleep(0.15)

**Note** that the timing is here under python's control, which is not very precise. The `Bundler` class allows to do better.

**Remarks**:

* note that the python code returns immediately and all events remain in scsynth
* note that unfortunately scsynth has a limited buffer for OSC messages, so it is not viable to spawn thousends of events. scsynth will then simply reject OSC messages.
* this sc3-specific problem motivated (and has been solved with) TimedQueue, see below.

### Bundles

In [14]:
from sc3nb.osc.osc_communication import Bundler

To send a single or multiple message(s) with a timetag as an OSC Bundle, you can use the `Bundler` class

* Bundlers allow to specify a timetag and thus let scsynth control the timing, which is much better, if applicable.
* A Bundler can be created as documented here

In [15]:
Bundler?

The prefered way of creating Bundlers for sending to the server is via

In [16]:
help(sc.server.bundler)

Help on method bundler in module sc3nb.sc_objects.server:

bundler(timetag=0, msg=None, msg_params=None, send_on_exit=True) method of sc3nb.sc_objects.server.SCServer instance
    Generate a Bundler with added server latency.
    
    This allows the user to easly add messages/bundles and send it.
    
    Parameters
    ----------
    timetag : float
        Time at which bundle content should be executed.
        This servers latency will be added upon this.
        If timetag <= 1e6 it is added to time.time().
    msg_addr : str
        SuperCollider address.
    msg_params : list, optional
        List of parameters to add to message.
         (Default value = None)
    
    Returns
    -------
    Bundler
        bundler for OSC bundling.



This will add the `sc.server.latency` time to the timetag. By default this is `0.0` but you can set it.

In [17]:
sc.server.latency

0.0

In [18]:
sc.server.latency = 0.1
sc.server.latency

0.1

A Bundler lets you add Messages with

In [19]:
help(Bundler.add)

Help on function add in module sc3nb.osc.osc_communication:

add(self, *args) -> 'Bundler'
    Add content to this Bundler.
    
    Parameters
    ----------
    args : OSCMessage or Bundler or Bundler arguments like
           (timetag, msg_addr, msg_params)
           (timetag, msg_addr)
           (timetag, msg)
    
    Returns
    -------
    Bundler
        self for chaining



In [20]:
msg1 = scn.OSCMessage("/s_new", ["s2", -1, 1, 1,])
msg2 = scn.OSCMessage("/n_free", [-1])
sc.server.bundler().add(1.5, msg1).add(1.9, msg2).send() # sound starts in 1.5s

Simpler is the usage of the *context manager*. This means you can use the `with` statement for better handling as follows: 

In [21]:
with sc.server.bundler() as bundler:
    bundler.add(0.0, msg1)
    bundler.add(0.3, msg2)

Instead of declaring the time explicitly with add you can also use `wait`

In [22]:
with sc.server.bundler() as bundler:
    for i in range(3):
        bundler.add(msg1)
        bundler.wait(0.3)
        bundler.add(msg2)
        bundler.wait(0.1)

Here are some different styles of coding the same sound with the Bundler features 

 - server Bundler with add

In [23]:
with sc.server.bundler(send_on_exit=False) as bundler:
    bundler.add(0.0, "/s_new", ["s2", -1, 1, 1,])
    bundler.add(0.3, "/n_free", [-1])

In [24]:
dg1 = bundler.to_raw_osc(0.0)  # we set the time_offset explicitly so all Bundle datagrams are the same
dg1

b'#bundle\x00\x83\xaa~\x80\x19\x99\x98\x00\x00\x00\x004#bundle\x00\x83\xaa~\x80\x19\x99\x98\x00\x00\x00\x00 /s_new\x00\x00,siii\x00\x00\x00s2\x00\x00\xff\xff\xff\xff\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00$#bundle\x00\x83\xaa~\x80ffh\x00\x00\x00\x00\x10/n_free\x00,i\x00\x00\xff\xff\xff\xff'

 - Bundler with explict latency set and using add

In [25]:
with Bundler(sc.server.latency, send_on_exit=False) as bundler:
    bundler.add(0.0, "/s_new", ["s2", -1, 1, 1])
    bundler.add(0.3, "/n_free", [-1])

In [26]:
dg2 = bundler.to_raw_osc(0.0)
dg2

b'#bundle\x00\x83\xaa~\x80\x19\x99\x98\x00\x00\x00\x004#bundle\x00\x83\xaa~\x80\x19\x99\x98\x00\x00\x00\x00 /s_new\x00\x00,siii\x00\x00\x00s2\x00\x00\xff\xff\xff\xff\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00$#bundle\x00\x83\xaa~\x80ffh\x00\x00\x00\x00\x10/n_free\x00,i\x00\x00\xff\xff\xff\xff'

 - server Bundler with implicit latency and using bundled messages

In [27]:
with sc.server.bundler(send_on_exit=False) as bundler:
    sc.server.msg("/s_new", ["s2", -1, 1, 1,], bundle=True)
    bundler.wait(0.3)
    sc.server.msg("/n_free", [-1], bundle=True)

In [28]:
dg3 = bundler.to_raw_osc(0.0)
dg3

b'#bundle\x00\x83\xaa~\x80\x19\x99\x98\x00\x00\x00\x004#bundle\x00\x83\xaa~\x80\x19\x99\x98\x00\x00\x00\x00 /s_new\x00\x00,siii\x00\x00\x00s2\x00\x00\xff\xff\xff\xff\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00$#bundle\x00\x83\xaa~\x80ffh\x00\x00\x00\x00\x10/n_free\x00,i\x00\x00\xff\xff\xff\xff'

In [29]:
assert dg1 == dg2 and dg1 == dg2, "The datagrams are not the same"

Note that you can use the Bundler with the Synth and Group classes for easier Message creation.

Also make sure to look at the [Automatic Bundling Feature](#Automatic-Bundling) which is using the bundled messages (`msg(..., bundle=True`)

In [30]:
t0 = time.time()
with sc.server.bundler() as bundler:
    for i, r in enumerate(np.random.randn(100)):
        onset = t0 + 3 + r
        freq = 500 + 5 * i
        bundler.add(onset, scn.Synth("s1", 
            {"freq": freq, "dur": 1.5, "num": abs(r)+1}, new=False
        ).new(return_msg=True))

#### Bundler Timestamp

Small numbers (<1e6) are interpreted as times in seconds relative to `time.time()`, evaluated at the time of sending

In [31]:
sc.server.bundler(0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 0.5s
sc.server.bundler(1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.0s

**Attention:** 

Sending bundles with relative times could lead to unprecise timings.
If you care about precision

* use a bundler with multiple messages (if you care about the timings relative to each other in one Bundler)
    * because all relative times of the inner messages are calculated on top of the outermost bundler timetag
* or provide an explict timetag (>1e6) to specify absolute times (see the following examples)


A single Bundler with multiple messages

In [32]:
bundler = sc.server.bundler()
bundler.add(0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1])
bundler.add(1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1])
bundler.send()  # second tone starts in 1.0s

using `time.time()+timeoffset` for absolute times

In [33]:
t0 = time.time()
sc.server.bundler(t0 + 0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 0.5s
sc.server.bundler(t0 + 1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.0s

In [34]:
t0 = time.time()
with sc.server.bundler() as bundler:
    for i, r in enumerate(np.random.randn(100)): # note: 1000 will give: msg too long
        onset = t0 + 3 + r
        freq = 500 + 5 * i
        msg_params = ["s1", -1, 1, 0, "freq", freq, "dur", 1.5, "num", abs(r)+1]
        bundler.add(onset, "/s_new", msg_params)

#### Nesting Bundlers

You can nest Bundlers: this will recalculate the time relative to the sending time of the outermost bundler

In [35]:
with sc.server.bundler() as bundler_outer:
    with sc.server.bundler() as bundler:
        sc.server.msg("/s_new", ["s2", 3333, 1, 1,], bundle=True)
        bundler.wait(0.3)
        sc.server.msg("/n_free", [3333], bundle=True)
    bundler_outer.wait(0.4)
    bundler_outer.add(bundler)

### Automatic Bundling

Probably the most convenient way of sending OSC is by using the Automatic Bundling Feature.

This allows you to simply use the SuperCollider Objects Synth and Group in the Context Manager of a Bundler and they will be automatically captured and stored. 

In [36]:
with sc.server.bundler() as bundler:
    synth = scn.Synth("s2")
    bundler.wait(0.3)
    synth.set("freq", 1000)
    bundler.wait(0.1)
    synth.free()
synth.wait()

**Note** that it is important that to *only* wait on the Synth *after* the context of the Bundler has been closed.

If you'd call `synth.wait()` in the Bundler context, it would wait before sending the /s_new Message to the server and then wait forever (or until timeout) for the /n_end notification.

In [37]:
try:
    with sc.server.bundler() as bundler:
        synth = scn.Synth("s2")
        bundler.wait(0.3)
        synth.set("freq", 1000)
        bundler.wait(0.1)
        synth.free()
        synth.wait(timeout=2)  # without a timeout this would hang forever
except RuntimeError as error:
    print(error)

Aborting. Exception raised in bundler: TimeoutError Timed out waiting for synth.


## Receiving OSC packets

sc3nb is receiving OSC messages with the help of queues, one AddressQueue for each OSC address for which we want to receive messages.

In [38]:
sc.server.msg_queues

{<MasterControlReply.STATUS_REPLY: '/status.reply'>: AddressQueue /status.reply : [],
 <MasterControlReply.SYNCED: '/synced'>: AddressQueue /synced : [],
 <MasterControlReply.VERSION_REPLY: '/version.reply'>: AddressQueue /version.reply : [],
 <NodeCommand.SET: '/n_set'>: AddressQueue /n_set : [],
 <NodeCommand.SETN: '/n_setn'>: AddressQueue /n_setn : [],
 <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>: AddressQueue /g_queryTree.reply : [],
 <NodeReply.INFO: '/n_info'>: AddressQueue /n_info : [],
 <BufferReply.INFO: '/b_info'>: AddressQueue /b_info : [],
 <BufferCommand.SET: '/b_set'>: AddressQueue /b_set : [],
 <BufferCommand.SETN: '/b_setn'>: AddressQueue /b_setn : [],
 <ControlBusCommand.SET: '/c_set'>: AddressQueue /c_set : [],
 <ControlBusCommand.SETN: '/c_setn'>: AddressQueue /c_setn : [],
 <ReplyAddress.RETURN_ADDR: '/return'>: AddressQueue /return : [],
 '/done/quit': AddressQueue /quit : [],
 '/done/notify': AddressQueue /notify : [],
 '/done/d_recv': AddressQueue /d_recv

To see more information what messages are sent and received, set the logging level to INFO as demonstrated below.

In [39]:
import logging
logging.basicConfig(level=logging.INFO)
# even more verbose logging is avaible via
# logging.basicConfig(level=logging.DEBUG)

### Getting replies

For certain outgoing OSC messages an incoming Message is defined.

This means that on sending such a message sc3nb automatically waits for the incoming message at the corresponding Queue and returns the result.

An example for this is `/sync {sync_id} -> /synced {sync_id}` 

In [40]:
sc.server.msg("/sync", 12345)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 12345)


12345

See all `(outgoing message, incoming message)` pairs:

In [41]:
sc.server.reply_addresses

{<MasterControlCommand.STATUS: '/status'>: <MasterControlReply.STATUS_REPLY: '/status.reply'>,
 <MasterControlCommand.SYNC: '/sync'>: <MasterControlReply.SYNCED: '/synced'>,
 <MasterControlCommand.VERSION: '/version'>: <MasterControlReply.VERSION_REPLY: '/version.reply'>,
 <SynthCommand.S_GET: '/s_get'>: <NodeCommand.SET: '/n_set'>,
 <SynthCommand.S_GETN: '/s_getn'>: <NodeCommand.SETN: '/n_setn'>,
 <GroupCommand.QUERY_TREE: '/g_queryTree'>: <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>,
 <NodeCommand.QUERY: '/n_query'>: <NodeReply.INFO: '/n_info'>,
 <BufferCommand.QUERY: '/b_query'>: <BufferReply.INFO: '/b_info'>,
 <BufferCommand.GET: '/b_get'>: <BufferCommand.SET: '/b_set'>,
 <BufferCommand.GETN: '/b_getn'>: <BufferCommand.SETN: '/b_setn'>,
 <ControlBusCommand.GET: '/c_get'>: <ControlBusCommand.SET: '/c_set'>,
 <ControlBusCommand.GETN: '/c_getn'>: <ControlBusCommand.SETN: '/c_setn'>,
 <MasterControlCommand.QUIT: '/quit'>: '/done/quit',
 <MasterControlCommand.NOTIFY: '/notify'>: 

You can get the reply address via

In [42]:
sc.server.get_reply_address("/sync")

<MasterControlReply.SYNCED: '/synced'>

or

In [43]:
sc.server.reply_addresses["/sync"]

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20054, 67108865, 20083, -1, 0)


<MasterControlReply.SYNCED: '/synced'>

If we specify `await_reply=False` the message will be kept in the queue

In [44]:
sc.server.msg("/sync", 1, await_reply=False)

In [45]:
sc.server.msg_queues[sc.server.get_reply_address("/sync")]

AddressQueue /synced : []

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20016, 67108865, -1, 20059, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 1)


In [46]:
sc.server.msg("/sync", 2, await_reply=False)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20068, 67108865, -1, 20016, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 2)


In [47]:
sc.server.msg_queues[sc.server.reply_addresses["/sync"]]

AddressQueue /synced : [1, 2]

In [48]:
sc.server.msg_queues["/synced"]

AddressQueue /synced : [1, 2]

You can see how many values were hold.

In [49]:
sc.server.msg_queues["/synced"].skips

2

Notice that these hold messages will be skipped. 

In [50]:
sc.server.msg("/sync", 3, await_reply=True)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 3)


3

In [51]:
sc.server.msg_queues["/synced"]

AddressQueue /synced : []

Therefore you should retrieve them with `get` and set `skip=False` if you care for old values in the queue and dont want them to be skipped.

In [52]:
sc.server.msg("/sync", 42, await_reply=False)
sc.server.msg_queues["/synced"].get(skip=False)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 42)


42

### Custom Message Queues

If you want to get additional OSC Messages you need to create a custom MessageQueue

In [53]:
from sc3nb.osc.osc_communication import MessageQueue

In [54]:
help(MessageQueue)

Help on class MessageQueue in module sc3nb.osc.osc_communication:

class MessageQueue(MessageHandler)
 |  MessageQueue(address: str, preprocess: Union[Callable, NoneType] = None)
 |  
 |  Queue to retrieve OSC messages send to the corresponding OSC address
 |  
 |  Method resolution order:
 |      MessageQueue
 |      MessageHandler
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, address: str, preprocess: Union[Callable, NoneType] = None)
 |      Create a new AddressQueue
 |      
 |      Parameters
 |      ----------
 |      address : str
 |          OSC address for this queue
 |      preprocess : function, optional
 |          function that will be applied to the value before they are enqueued
 |           (Default value = None)
 |  
 |  get(self, timeout: float = 5, skip: bool = True) -> Any
 |      Returns a value from the queue
 |      
 |      Parameters
 |      ----------
 |      timeout : int, optional
 |          Time in seconds t

In [55]:
mq = MessageQueue("/test")

In [56]:
sc.server.add_msg_queue(mq)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20032, 67108865, -1, 20068, 0)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57110)


In [57]:
sc.server.msg("/test", ["Hi!"], receiver="sc3nb")

Traceback (most recent call last):
  File "C:\Users\Dennis\Anaconda3\envs\soni\lib\socketserver.py", line 650, in process_request_thread
    self.finish_request(request, client_address)
  File "C:\Users\Dennis\Anaconda3\envs\soni\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\Dennis\Anaconda3\envs\soni\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "C:\Users\Dennis\Anaconda3\envs\soni\lib\site-packages\pythonosc\osc_server.py", line 29, in handle
    self.server.dispatcher.call_handlers_for_packet(self.request[0], self.client_address)
  File "C:\Users\Dennis\Anaconda3\envs\soni\lib\site-packages\pythonosc\dispatcher.py", line 193, in call_handlers_for_packet
    for handler in handlers:
  File "C:\Users\Dennis\Anaconda3\envs\soni\lib\site-packages\pythonosc\dispatcher.py", line 161, in handlers_for_address
    for addr, handlers in self._map.items():
RuntimeError: dictionary changed si

INFO:sc3nb.sc_objects.server:OSC msg received from sc3nb: ('/test', 'Hi!')


In [58]:
sc.server.msg_queues["/test"]

AddressQueue /test : ['Hi!']

In [59]:
sc.server.msg("/test", ["Hello!"], receiver="sc3nb")

INFO:sc3nb.sc_objects.server:OSC msg received from sc3nb: ('/test', 'Hello!')


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20083, 67108865, 20030, -1, 0)


In [60]:
sc.server.msg_queues["/test"]

AddressQueue /test : ['Hi!', 'Hello!']

In [61]:
sc.server.msg_queues["/test"].get()

'Hi!'

In [62]:
sc.server.msg_queues["/test"].get()

'Hello!'

If you want to create a pair of an outgoing message that will receive a certain incomming message you need to specify it via the `out_addr` arugment of `add_msg_queue` or you could use the shortcut for this `add_msg_pairs`

In [63]:
help(sc.server.add_msg_pairs)

Help on method add_msg_pairs in module sc3nb.osc.osc_communication:

add_msg_pairs(msg_pairs: Dict[str, str]) -> None method of sc3nb.sc_objects.server.SCServer instance
    Add the provided pairs for message receiving.
    
    Parameters
    ----------
    msg_pairs : dict[str, str], optional
        dict containing user specified message pairs.
        {msg_addr: reply_addr}



INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20030, 67108865, 20002, -1, 0)


In [64]:
sc.server.add_msg_pairs({"/hi": "/hi.reply"})

Let's use `OSCdef` in sclang to send us replies.

In [65]:
%%sc
OSCdef.newMatching("say_hi", {|msg, time, addr, recvPort| addr.sendMsg("/hi.reply", "Hello there!")}, '/hi');

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20076, 67108865, -1, 20032, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20002, 67108865, 20067, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20067, 67108865, 20045, -1, 0)


-> OSCdef(say_hi, /hi, nil, nil, nil)


In [66]:
sc.server.msg("/hi", receiver="sclang")

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20045, 67108865, 20011, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/hi.reply', 'Hello there!')


'Hello there!'

There is also the class MessageQueueCollection, which allows to create multiple MessageQueues for a multiple `address/subaddresses` combination

In [67]:
from sc3nb.osc.osc_communication import MessageQueueCollection

In [68]:
help(MessageQueueCollection)

Help on class MessageQueueCollection in module sc3nb.osc.osc_communication:

class MessageQueueCollection(MessageHandler)
 |  MessageQueueCollection(address: str, sub_addrs: Union[Sequence[str], NoneType] = None)
 |  
 |  A collection of MessageQueues that are all sent to one and the same first address.
 |  
 |  Method resolution order:
 |      MessageQueueCollection
 |      MessageHandler
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, item) -> bool
 |  
 |  __getitem__(self, key)
 |  
 |  __init__(self, address: str, sub_addrs: Union[Sequence[str], NoneType] = None)
 |      Create a collection of MessageQueues under the same first address
 |      
 |      Parameters
 |      ----------
 |      address : str
 |          first message address that is the same for all MessageQueues
 |      sub_addrs : Optional[Sequence[str]], optional
 |          secound message addresses with seperate queues, by default None
 |          Additional Messa

In [69]:
mqc = MessageQueueCollection("/collect", ["/address1", "/address2"])
sc.server.add_msg_queue_collection(mqc)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20037, 67108865, -1, 20076, 0)


In [70]:
mqc = MessageQueueCollection("/auto_collect")
sc.server.add_msg_queue_collection(mqc)

In [71]:
sc.server.reply_addresses

{<MasterControlCommand.STATUS: '/status'>: <MasterControlReply.STATUS_REPLY: '/status.reply'>,
 <MasterControlCommand.SYNC: '/sync'>: <MasterControlReply.SYNCED: '/synced'>,
 <MasterControlCommand.VERSION: '/version'>: <MasterControlReply.VERSION_REPLY: '/version.reply'>,
 <SynthCommand.S_GET: '/s_get'>: <NodeCommand.SET: '/n_set'>,
 <SynthCommand.S_GETN: '/s_getn'>: <NodeCommand.SETN: '/n_setn'>,
 <GroupCommand.QUERY_TREE: '/g_queryTree'>: <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>,
 <NodeCommand.QUERY: '/n_query'>: <NodeReply.INFO: '/n_info'>,
 <BufferCommand.QUERY: '/b_query'>: <BufferReply.INFO: '/b_info'>,
 <BufferCommand.GET: '/b_get'>: <BufferCommand.SET: '/b_set'>,
 <BufferCommand.GETN: '/b_getn'>: <BufferCommand.SETN: '/b_setn'>,
 <ControlBusCommand.GET: '/c_get'>: <ControlBusCommand.SET: '/c_set'>,
 <ControlBusCommand.GETN: '/c_getn'>: <ControlBusCommand.SETN: '/c_setn'>,
 <MasterControlCommand.QUIT: '/quit'>: '/done/quit',
 <MasterControlCommand.NOTIFY: '/notify'>: 

In [72]:
sc.server.msg_queues

{<MasterControlReply.STATUS_REPLY: '/status.reply'>: AddressQueue /status.reply : [],
 <MasterControlReply.SYNCED: '/synced'>: AddressQueue /synced : [],
 <MasterControlReply.VERSION_REPLY: '/version.reply'>: AddressQueue /version.reply : [],
 <NodeCommand.SET: '/n_set'>: AddressQueue /n_set : [],
 <NodeCommand.SETN: '/n_setn'>: AddressQueue /n_setn : [],
 <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>: AddressQueue /g_queryTree.reply : [],
 <NodeReply.INFO: '/n_info'>: AddressQueue /n_info : [],
 <BufferReply.INFO: '/b_info'>: AddressQueue /b_info : [],
 <BufferCommand.SET: '/b_set'>: AddressQueue /b_set : [],
 <BufferCommand.SETN: '/b_setn'>: AddressQueue /b_setn : [],
 <ControlBusCommand.SET: '/c_set'>: AddressQueue /c_set : [],
 <ControlBusCommand.SETN: '/c_setn'>: AddressQueue /c_setn : [],
 <ReplyAddress.RETURN_ADDR: '/return'>: AddressQueue /return : [],
 '/done/quit': AddressQueue /quit : [],
 '/done/notify': AddressQueue /notify : [],
 '/done/d_recv': AddressQueue /d_recv

In [73]:
%%scv
OSCdef.newMatching("ab", {|msg, time, addr, recvPort| addr.sendMsg('/collect', '/address1', "toast".scramble)}, '/address1');

-> OSCdef(ab, /address1, nil, nil, nil)


In [74]:
%%scv
OSCdef.newMatching("ab", {|msg, time, addr, recvPort| addr.sendMsg('/collect', '/address2', "sonification".scramble)}, '/address2');

-> OSCdef(ab, /address2, nil, nil, nil)


In [75]:
sc.server.msg("/address1", receiver="sclang")

INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/collect', '/address1', 'saott')


'saott'

In [76]:
sc.server.msg("/address2", receiver="sclang")

INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/collect', '/address2', 'iontnosaciif')


'iontnosaciif'

## Examples 

### Creating an OSC responder and msg to sclang for synthesis

In [77]:
%%sc
OSCdef(\dinger, { | msg, time, addr, recvPort |
    var freq = msg[2];
    {Pulse.ar(freq, 0.04, 0.3)!2 * EnvGen.ar(Env.perc, doneAction:2)}.play()
}, '/ding')

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20001, 67108865, -1, 20037, 0)


-> OSCdef(dinger, /ding, nil, nil, nil)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20061, 67108865, -1, 20001, 0)


In [78]:
with scn.Bundler(receiver=sc.lang.addr):
    for i in range(5):
        sc.server.msg("/ding", ["freq", 1000-5*i], bundle=True)

INFO:sc3nb.osc.osc_communication:send to sclang : <Bundler {0.0: [<OSCMessage("/ding", ['freq', 1000])>, <OSCMessage("/ding", ['freq', 995])>, <OSCMessage("/ding", ['freq', 990])>, <OSCMessage("/ding", ['freq', 985])>, <OSCMessage("/ding", ['freq', 980])>]}> contents size 5 


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20011, 67108865, 20066, -1, 0)


In [79]:
for i in range(5):
    sc.server.msg("/ding", ["freq", 1000-5*i], receiver=sc.lang.addr)

In [80]:
%scv OSCdef.freeAll()

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1000, 1, -1, -336, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1001, 1, -1, 1000, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1002, 1, -1, 1001, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1003, 1, -1, 1002, 0)


-> OSCdef


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1004, 1, -1, 1003, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1005, 1, -1, 1004, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1006, 1, -1, 1005, 0)


In [81]:
sc.exit()

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20072, 67108865, -1, 20061, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1007, 1, -1, 1006, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1008, 1, -1, 1007, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1009, 1, -1, 1008, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/done', '/quit')
